## Data loading and exploration

In [1]:
# Load libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [3]:
# Path to course data folder
data_folder_fp = os.path.join('/',
                              'courses',
                              'EDS220', 
                              'data')

# Make file path to data
fp = os.path.join(data_folder_fp, 
                  'arctic_communities.geojson')
df = gpd.read_file(fp)

# View changes
df.head()

,admin,country,n_communities,geometry
0,United States of America,US,115,"MULTIPOLYGON (((-132.74687 56.52568, -132.7576..."
1,United Kingdom,GB,96,"MULTIPOLYGON (((-2.66768 51.62300, -2.74214 51..."
2,Sweden,SE,133,"MULTIPOLYGON (((19.07646 57.83594, 18.99375 57..."
3,Russia,RU,774,"MULTIPOLYGON (((145.88154 43.45952, 145.89561 ..."
4,Norway,NO,48,"MULTIPOLYGON (((20.62217 69.03687, 20.49199 69..."


## Check geometry types

In [6]:
# Check geometry type of columns
df.geom_type

0     MultiPolygon
1     MultiPolygon
2     MultiPolygon
3     MultiPolygon
4     MultiPolygon
5     MultiPolygon
6          Polygon
7          Polygon
8     MultiPolygon
9     MultiPolygon
10    MultiPolygon
11    MultiPolygon
12    MultiPolygon
13    MultiPolygon
14         Polygon
dtype: object

In [32]:
if (df.geom_type == 'polygon').all():
    print('All geometryies are Polygons')
else:
    print(f"Multiple feature types: {df.geom_type.unique()}")

Multiple feature types: ['MultiPolygon' 'Polygon']


In [33]:
def check_polygons(geodf):
    """
    Checking the geometry of each column in a GeoDataFrame
    Args: Geodf: the geodataframe
    Returns: Str: Polygon types
    """
    if (geodf.geom_type == 'Polygon').all():
        return('All geometryies are polygons')
    else:
        return(f"Multiple feature types: {geodf.geom_type.unique()}")

In [34]:
check_polygons(df)

"Multiple feature types: ['MultiPolygon' 'Polygon']"

## Explode polygons

In [37]:
df = df.explode(index_parts = False).reset_index()

# Check
check_polygons(df)

ValueError: cannot insert level_0, already exists

## Compute minimum y-coordinate for polygons

In [43]:
# Select first row
df.iloc[[0]] # makes it a df

# Select geom
type(df.iloc[[0]])

# Select geom
df.iloc[[0]]['geometry']

# Check geom type
type(df.iloc[[0]]['geometry'])

# Use bounds
df.iloc[[0]]['geometry'].bounds['miny']

0    56.511035
Name: miny, dtype: float64

In [46]:
# Make function miny
def min_y(geodf_row):
    return geodf_row.geometry.bounds[1] # return second position

# Use function to make new column
df['miny'] = df.apply(min_y, axis=1) # to go over rows

df

,level_0,index,admin,country,n_communities,geometry,miny
0,0,0,United States of America,US,115,"POLYGON ((-132.74687 56.52568, -132.75762 56.5...",56.511035
1,1,0,United States of America,US,115,"POLYGON ((-132.77988 56.24727, -132.83096 56.2...",56.244141
2,2,0,United States of America,US,115,"POLYGON ((-134.31274 58.22891, -134.31987 58.2...",58.204102
3,3,0,United States of America,US,115,"POLYGON ((-145.11851 60.33711, -145.15049 60.3...",60.312646
4,4,0,United States of America,US,115,"POLYGON ((-144.56563 59.81841, -144.61357 59.8...",59.812646
...,...,...,...,...,...,...,...
476,476,13,Canada,CA,7,"POLYGON ((-109.16641 67.98237, -109.05391 67.9...",67.878809
477,477,13,Canada,CA,7,"POLYGON ((-108.09272 67.00518, -107.96646 66.9...",66.857812
478,478,13,Canada,CA,7,"POLYGON ((-109.32314 67.99087, -109.36084 67.9...",67.987598
479,479,13,Canada,CA,7,"POLYGON ((-139.04312 69.57690, -139.12573 69.5...",69.539307


In [ ]:
# select polygons
df[df['min_y'] >= 40]

# Update crs
df = df[df['min_y'] >= 40].to_crs('EPSG:3413')
df.plot()